In [2]:
import pandas as pd #the csv reading library
import matplotlib.pyplot as plt #the ploting library
import glob #to get file names
import numpy as np
from pandas import Series
import seaborn as sns
from matplotlib.pyplot import figure
import cmocean.cm as cmo
import seaborn as sns

from pylab import rcParams
from numpy import diff
from scipy import interpolate
import scipy.stats as stats

from scipy.signal import savgol_filter
from sklearn.linear_model import LinearRegression

import statsmodels.api as sm
from statsmodels.stats.anova import anova_lm
from statsmodels.formula.api import ols

import matplotlib.gridspec as gridspec

import cmocean
import cmocean.cm as cmo
import seaborn as sns

from scipy.interpolate import interp1d

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

In [4]:
mds_fe2_rz = pd.read_csv('../Cruise_2024/Data/FeT_RZ.csv', sep=',', index_col=0)
mds_fet_rz = pd.read_csv('../Cruise_2024/Data/Fe2_RZ.csv', sep=',', index_col=0)

mds_toc = pd.read_csv('../Cruise_2024/Data/TOC_MDS_Depths.csv', sep=',', index_col=0)
mds_toc_join = mds_toc['NPOC_uM']
mds_toc_joinLocation = mds_toc['Location']


mds_fe_rz = mds_fe2_rz.join(mds_fet_rz)
mds_fe_rz['Fe3_uM'] = mds_fe_rz['FeT_uM'] - mds_fe_rz['Fe2_uM']
mds_fe_rz.loc[mds_fe_rz['Fe3_uM'] < 0, 'Fe3_uM'] = np.nan


mds_fe_pw = pd.read_csv('../Cruise_2024/Data/PW_MSD_24.csv', sep=',', index_col=0)
mds_fe_pw['FeT_uM'] = mds_fe_pw['FeT_um'] 
mds_fe_pw['Fe2_uM'] = mds_fe_pw['Fe2'] 
mds_fe_pw['Fe3_uM'] = mds_fe_pw['Fe3'] 
mds_fe_pw.loc[mds_fe_pw['Fe3_uM'] < 0, 'Fe3_uM'] = np.nan


mds_fe_pw = mds_fe_pw[['FeT_uM', 'Fe2_uM', 'Fe3_uM']]
mds_fe_rz = mds_fe_rz[['FeT_uM', 'Fe2_uM', 'Fe3_uM']]

mds_pw = mds_fe_pw.join(mds_toc)

mds_rz = mds_fe_rz.join(mds_toc)

mds_rz['Type'] = 'Rhz'
mds_pw['Type'] = 'Cent'

mds_rz['Depth'] = pd.to_numeric(mds_rz['Depth'], errors='coerce')
mds_rz = mds_rz.sort_values(by='Depth')

#mds_pw = mds_pw[['Fe3_uM', 'a350', 'Type', 'NPOC_uM']]
mds_rz

,FeT_uM,Fe2_uM,Fe3_uM,NISK_Note,Treatment,Campaign,Date,Location,Depth,NPOC_mg.L,TN_mg.L,NPOC_uM,TN_uM,Type,Depth_cm
Sample_ID,,,,,,,,,,,,,,,
241060,0.446000,-0.034,0.480000,NaN,P2F,MDS,7/24/2024,CL5,0.0,3.497923,0.762542,291.493590,54.467253,Rhz,NaN
241860,0.402607,-0.047,0.449607,NaN,P2F,MDS,7/28/2024,N,0.0,2.841456,0.471416,236.787963,33.672540,Rhz,NaN
241760,3.197798,-0.198,3.395798,NaN,P2F,MDS,7/29/2024,D,0.0,2.970317,0.536558,247.526389,38.325595,Rhz,NaN
241660,-1.049000,0.135,NaN,NaN,P2F,MSD,7/31/2024,I,0.0,4.593050,0.661485,382.754167,47.248929,Rhz,NaN
241460,0.563840,0.495,0.068840,NaN,P2F,MDS,7/25/2024,CL13,0.0,3.557400,0.840718,296.450000,60.051250,Rhz,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241372,1.207454,0.729,0.478454,NaN,P2F,MDS,7/21/2024,CL11,21.8,9.145831,1.244338,762.152564,88.881319,Rhz,NaN
241362,6.188338,6.094,0.094338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rhz,NaN
241368,2.411074,1.383,1.028074,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rhz,NaN


In [10]:
interpolated_data = []

codes_list = df_por['code'].unique()

for code in codes_list:
    data_iron = df_iron_vol[df_iron_vol['code'] == code]
    data_por = df_por[df_por['code'] == code]
    depth_fe_max = data_iron['Depth'].max()
    depth_fe_min = data_iron['Depth'].min()
    depth_po_max = data_por['depth'].max()
    depth_po_min = data_por['depth'].min()
    
    data_iron_2 = data_iron
    data_iron = data_iron.loc[data_iron['Depth'] >= depth_po_min]
    data_iron = data_iron.loc[data_iron['Depth'] <= depth_po_max] 
    
    depth = data_iron['Depth']
    depth = np.asarray(depth, dtype=float).ravel()
    fe2 = data_iron['Fe2_uM']
    fe2 = np.asarray(fe2, dtype=float).ravel()
    fe3 = data_iron['Fe3_uM']
    fe3 = np.asarray(fe3, dtype=float).ravel()
    fet = data_iron['FeT_uM']
    fet = np.asarray(fet, dtype=float).ravel()
    stn = data_iron['Location']
    camp = data_iron['Campaign']
    
    depth_p = data_por['depth']
    depth_p = np.asarray(depth_p, dtype=float).ravel()
    porosity = data_por['porosity']
    porosity = np.asarray(porosity, dtype=float).ravel()
    
    valid_indices = np.isfinite(porosity)
    porosity = porosity[valid_indices]
    depth_p = depth_p[valid_indices]
    
    cs = interp1d(depth_p, porosity)
    por_interpolated = cs(depth)
    
    interpolated_data.append(pd.DataFrame({'por_interpolated': por_interpolated,
                                           'Depth': depth,
                                           'fe2': fe2,
                                           'fe3': fe3,
                                           'fet': fet,
                                           'code': code,
                                           'stn': stn, 
                                           'camp': camp
                                          }))

interpolated_df = pd.concat(interpolated_data, ignore_index=True)
interpolated_df.dropna(subset=['Depth'], inplace=True)

In [12]:
df_cal = interpolated_df
df_cal['sed_density_Gpercm3'] = 2.6

#fe uM to M 
df_cal['fe2_mol_per_m3'] =  df_cal['fe2'] / 1000000 * 1000
df_cal['fe3_mol_per_m3'] =  df_cal['fe3'] / 1000000 * 1000
df_cal['fet_mol_per_m3'] =  df_cal['fet'] / 1000000 * 1000

df_cal['fe2_mol_per_m3_vol'] =  df_cal['fe2_mol_per_m3'] * df_cal['por_interpolated']
df_cal['fe3_mol_per_m3_vol'] =  df_cal['fe3_mol_per_m3'] * df_cal['por_interpolated']
df_cal['fet_mol_per_m3_vol'] =  df_cal['fet_mol_per_m3'] * df_cal['por_interpolated']

df_cal['depth_m'] = df_cal['Depth'] / 100

df_cal

,por_interpolated,Depth,fe2,fe3,fet,code,stn,camp,sed_density_Gpercm3,fe2_mol_per_m3,fe3_mol_per_m3,fet_mol_per_m3,fe2_mol_per_m3_vol,fe3_mol_per_m3_vol,fet_mol_per_m3_vol,depth_m
0,0.897196,0.35,0.145,NaN,NaN,St.11_GOM2021,St.11,GOM2021,2.6,0.000145,NaN,NaN,0.000130,NaN,NaN,0.0035
1,0.891846,1.05,NaN,NaN,NaN,St.11_GOM2021,St.11,GOM2021,2.6,NaN,NaN,NaN,NaN,NaN,NaN,0.0105
2,0.879845,1.75,0.145,NaN,NaN,St.11_GOM2021,St.11,GOM2021,2.6,0.000145,NaN,NaN,0.000128,NaN,NaN,0.0175
3,0.870578,2.45,0.145,NaN,NaN,St.11_GOM2021,St.11,GOM2021,2.6,0.000145,NaN,NaN,0.000126,NaN,NaN,0.0245
4,0.864441,3.15,0.145,NaN,NaN,St.11_GOM2021,St.11,GOM2021,2.6,0.000145,NaN,NaN,0.000125,NaN,NaN,0.0315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779,0.699232,20.20,0.379,4.384347,4.763347,St.MK_GOM22SU,St.MK,GOM22SU,2.6,0.000379,0.004384,0.004763,0.000265,0.003066,0.003331,0.2020
780,0.704056,22.20,0.699,9.290839,9.989839,St.MK_GOM22SU,St.MK,GOM22SU,2.6,0.000699,0.009291,0.009990,0.000492,0.006541,0.007033,0.2220
781,0.659089,24.70,0.677,8.786629,9.463629,St.MK_GOM22SU,St.MK,GOM22SU,2.6,0.000677,0.008787,0.009464,0.000446,0.005791,0.006237,0.2470
782,0.605461,27.70,1.032,NaN,NaN,St.MK_GOM22SU,St.MK,GOM22SU,2.6,0.001032,NaN,NaN,0.000625,NaN,NaN,0.2770


In [13]:
Frames = []

for code in codes_list: 
    df_play = df_cal[df_cal['code'] == code] 

    # Create a row with NaN values
    new_row = pd.DataFrame([np.nan] * len(df_play.columns)).T

    # Set column names for the new row
    new_row.columns = df_play.columns
    new_row['depth_m'] = 0 
    new_row['fet_mol_per_m3_vol'] = 0 
    new_row['fe2_mol_per_m3_vol'] = 0 
    new_row['fe3_mol_per_m3_vol'] = 0 

    # Append the new row to the DataFrame
    df_play = pd.concat([pd.DataFrame(new_row), df_play], ignore_index=True)

    ds_fet = df_play.dropna(subset=['depth_m', 'fet_mol_per_m3_vol'])
    ds_fe3 = df_play.dropna(subset=['depth_m', 'fe3_mol_per_m3_vol'])
    ds_fe2 = df_play.dropna(subset=['depth_m', 'fe2_mol_per_m3_vol'])


    # Define the depth range you want to integrate over
    start_depth = 0
    stop_depth = .05

    # Interpolating concentration at depth 5 using linear interpolation
    fet_at_depth_5 = np.interp(stop_depth, ds_fet['depth_m'], ds_fet['fet_mol_per_m3_vol'])
    fe3_at_depth_5 = np.interp(stop_depth, ds_fe3['depth_m'], ds_fe3['fe3_mol_per_m3_vol'])
    fe2_at_depth_5 = np.interp(stop_depth, ds_fe2['depth_m'], ds_fe2['fe2_mol_per_m3_vol'])

    #add he depth 5 data  
    new_row.columns = ds_fet.columns
    new_row['depth_m'] = stop_depth
    new_row['fet_mol_per_m3_vol'] = fet_at_depth_5
    new_row['fe3_mol_per_m3_vol'] = fe3_at_depth_5
    new_row['fe2_mol_per_m3_vol'] = fe2_at_depth_5


    # Selecting data within the specified depth range
    selected_data_fet = ds_fet[(ds_fet['depth_m'] >= start_depth) & (ds_fet['depth_m'] <= stop_depth)]
    selected_data_fet = pd.concat([selected_data_fet, new_row])

    selected_data_fe3 = ds_fe3[(ds_fe3['depth_m'] >= start_depth) & (ds_fe3['depth_m'] <= stop_depth)]
    selected_data_fe3 = pd.concat([selected_data_fe3, new_row])

    selected_data_fe2 = ds_fe2[(ds_fe2['depth_m'] >= start_depth) & (ds_fe2['depth_m'] <= stop_depth)]
    selected_data_fe2 = pd.concat([selected_data_fe2, new_row])

    # Calculating the integral using the trapezoidal rule
    integral_fet = np.trapz(selected_data_fet['fet_mol_per_m3_vol'], selected_data_fet['depth_m']) / stop_depth
    integral_fe3 = np.trapz(selected_data_fe3['fe3_mol_per_m3_vol'], selected_data_fe3['depth_m']) / stop_depth
    integral_fe2 = np.trapz(selected_data_fe2['fe2_mol_per_m3_vol'], selected_data_fe2['depth_m']) / stop_depth

    dic = {'code':[code],
           'fet_mol_per_m3_0_to_5cm':[integral_fet],
           'fe3_mol_per_m3_0_to_5cm':[integral_fe3], 
           'fe2_mol_per_m3_0_to_5cm':[integral_fe2]}
        
    slopes = pd.DataFrame(dic)
    slopes = slopes.set_index('code', drop=True)
    Frames.append(slopes)
    
stocks_pw_fe_5 = pd.concat(Frames, axis=0, ignore_index=False)

stocks_pw_fe_5

,fet_mol_per_m3_0_to_5cm,fe3_mol_per_m3_0_to_5cm,fe2_mol_per_m3_0_to_5cm
code,,,
St.11_GOM2021,0.000460,0.000395,0.000123
St.14_GOM2021,0.044729,0.038293,0.006435
St.15_GOM2021,0.019401,0.018704,0.000697
St.16_GOM2021,0.076588,0.064643,0.011944
St.2_GOM2021,0.052417,0.045372,0.007045
St.4_GOM2021,0.012621,0.011734,0.000871
St.5B_GOM2021,0.049136,0.043472,0.005664
St.7_GOM2021,0.043682,0.032189,0.011174
St.9_GOM2021,0.018908,0.015226,0.003682


In [14]:
Frames = []

for code in codes_list: 
    df_play = df_cal[df_cal['code'] == code] 

    # Create a row with NaN values
    new_row = pd.DataFrame([np.nan] * len(df_play.columns)).T

    # Set column names for the new row
    new_row.columns = df_play.columns
    new_row['depth_m'] = 0 
    new_row['fet_mol_per_m3_vol'] = 0 
    new_row['fe2_mol_per_m3_vol'] = 0 
    new_row['fe3_mol_per_m3_vol'] = 0 

    # Append the new row to the DataFrame
    df_play = pd.concat([pd.DataFrame(new_row), df_play], ignore_index=True)

    ds_fet = df_play.dropna(subset=['depth_m', 'fet_mol_per_m3_vol'])
    ds_fe3 = df_play.dropna(subset=['depth_m', 'fe3_mol_per_m3_vol'])
    ds_fe2 = df_play.dropna(subset=['depth_m', 'fe2_mol_per_m3_vol'])


    # Define the depth range you want to integrate over
    start_depth = 0
    stop_depth = .25

    # Interpolating concentration at depth 5 using linear interpolation
    fet_at_depth_5 = np.interp(stop_depth, ds_fet['depth_m'], ds_fet['fet_mol_per_m3_vol'])
    fe3_at_depth_5 = np.interp(stop_depth, ds_fe3['depth_m'], ds_fe3['fe3_mol_per_m3_vol'])
    fe2_at_depth_5 = np.interp(stop_depth, ds_fe2['depth_m'], ds_fe2['fe2_mol_per_m3_vol'])

    #add he depth 5 data  
    new_row.columns = ds_fet.columns
    new_row['depth_m'] = stop_depth
    new_row['fet_mol_per_m3_vol'] = fet_at_depth_5
    new_row['fe3_mol_per_m3_vol'] = fe3_at_depth_5
    new_row['fe2_mol_per_m3_vol'] = fe2_at_depth_5


    # Selecting data within the specified depth range
    selected_data_fet = ds_fet[(ds_fet['depth_m'] >= start_depth) & (ds_fet['depth_m'] <= stop_depth)]
    selected_data_fet = pd.concat([selected_data_fet, new_row])

    selected_data_fe3 = ds_fe3[(ds_fe3['depth_m'] >= start_depth) & (ds_fe3['depth_m'] <= stop_depth)]
    selected_data_fe3 = pd.concat([selected_data_fe3, new_row])

    selected_data_fe2 = ds_fe2[(ds_fe2['depth_m'] >= start_depth) & (ds_fe2['depth_m'] <= stop_depth)]
    selected_data_fe2 = pd.concat([selected_data_fe2, new_row])

    # Calculating the integral using the trapezoidal rule
    integral_fet = np.trapz(selected_data_fet['fet_mol_per_m3_vol'], selected_data_fet['depth_m']) / stop_depth
    integral_fe3 = np.trapz(selected_data_fe3['fe3_mol_per_m3_vol'], selected_data_fe3['depth_m']) / stop_depth
    integral_fe2 = np.trapz(selected_data_fe2['fe2_mol_per_m3_vol'], selected_data_fe2['depth_m']) / stop_depth

    dic = {'code':[code],
           'fet_mol_per_m3_0_to_25cm':[integral_fet],
           'fe3_mol_per_m3_0_to_25cm':[integral_fe3], 
           'fe2_mol_per_m3_0_to_25cm':[integral_fe2]}
        
    slopes = pd.DataFrame(dic)
    slopes = slopes.set_index('code', drop=True)
    Frames.append(slopes)
    
stocks_pw_fe_25 = pd.concat(Frames, axis=0, ignore_index=False)

stocks_pw_fe_25

,fet_mol_per_m3_0_to_25cm,fe3_mol_per_m3_0_to_25cm,fe2_mol_per_m3_0_to_25cm
code,,,
St.11_GOM2021,0.023253,0.022484,0.000780
St.14_GOM2021,0.030305,0.027952,0.002353
St.15_GOM2021,0.030869,0.028864,0.002005
St.16_GOM2021,0.055809,0.050931,0.004878
St.2_GOM2021,0.034416,0.031250,0.003166
St.4_GOM2021,0.006348,0.008427,0.000255
St.5B_GOM2021,0.022840,0.021003,0.001837
St.7_GOM2021,0.018852,0.015575,0.003214
St.9_GOM2021,0.046517,0.043031,0.003485


In [15]:
stocks_pw_fe = pd.concat([stocks_pw_fe_25, stocks_pw_fe_5], axis=1)
stocks_pw_fe.to_csv('../Dissertation_projects/Data/Stats_data_sets/Iron_stocks.csv')
stocks_pw_fe

,fet_mol_per_m3_0_to_25cm,fe3_mol_per_m3_0_to_25cm,fe2_mol_per_m3_0_to_25cm,fet_mol_per_m3_0_to_5cm,fe3_mol_per_m3_0_to_5cm,fe2_mol_per_m3_0_to_5cm
code,,,,,,
St.11_GOM2021,0.023253,0.022484,0.000780,0.000460,0.000395,0.000123
St.14_GOM2021,0.030305,0.027952,0.002353,0.044729,0.038293,0.006435
St.15_GOM2021,0.030869,0.028864,0.002005,0.019401,0.018704,0.000697
St.16_GOM2021,0.055809,0.050931,0.004878,0.076588,0.064643,0.011944
St.2_GOM2021,0.034416,0.031250,0.003166,0.052417,0.045372,0.007045
St.4_GOM2021,0.006348,0.008427,0.000255,0.012621,0.011734,0.000871
St.5B_GOM2021,0.022840,0.021003,0.001837,0.049136,0.043472,0.005664
St.7_GOM2021,0.018852,0.015575,0.003214,0.043682,0.032189,0.011174
St.9_GOM2021,0.046517,0.043031,0.003485,0.018908,0.015226,0.003682
